In [28]:
try:
  import google.colab
  IN_COLAB = True
except:
  IN_COLAB = False

# keras module for building LSTM 
from keras.layers import Embedding, LSTM, Dense, Dropout, Flatten, Bidirectional
from keras.preprocessing.text import Tokenizer
from keras.callbacks import EarlyStopping
from keras.models import Sequential
import keras.utils as ku 
import keras
import tensorflow as tf
from tensorflow.keras.utils import Sequence

if IN_COLAB:
  !pip install Keras-Preprocessing
from keras_preprocessing.sequence import pad_sequences
from keras.preprocessing.text import text_to_word_sequence

import pandas as pd
import numpy as np
import string, os 
import matplotlib.pyplot as plt
import seaborn as sns

In [29]:
# Helper to plot loss
def plot_loss(history):
  plt.plot(history.history['loss'], label='loss')
  plt.legend()
  plt.grid(True)
  plt.show()

def plot_acc(history):
  plt.plot(history.history['acc'], label='accuracy')
  plt.legend()
  plt.grid(True)
  plt.show()

# Text Generation with LSTM

As we have seen, LSTM models are excellent at dealing with sequential data. As luck would have it, text is also sequental data! We can train a model to predict the next word in a sentence, then use that smarts to generate new text. Basically ChatGPT, but far better. 

### Text for Training

We need some text from which to train our model to speak, I captured a small extract of text from Reddit posts, which vaguely resembles actual language. We'll first need to clean up our data a bit before we can assemble it for modelling. The inital cleaning bits are just like what we used in NLP, we just need to get rid of all the junk. 

We can use pretty much anything that you can imagine as source, and assuming we can gather enough data and train our model, the generated speech will be styled after the source. I liken it to going on vacation in Indonesia and talking to Indonesians who spoke English like Australian surfer bros - their training data was a little weird, so the output was a little weird too. If you're looking to build your best ChatGPT competitor you will want a lot of data, specifically a lot of data that is representative of the full gamut of how you want your model to write. If you want slang in the new text, you can't really train on Shakespeare and Wikipedia.

In [30]:
# Reddit WSB data
#train_text_file = keras.utils.get_file('train_text.txt', 'https://jrssbcrsefilesnait.blob.core.windows.net/3950data1/reddit_wsb.csv')
#USE_COL = "body"

# Complaint data
train_text_file = keras.utils.get_file('train_text3.txt', 'https://jrssbcrsefilesnait.blob.core.windows.net/3950data1/complaints_processed_clean.csv')
USE_COL = "narrative"

train_text = pd.read_csv(train_text_file)
train_text.sample(10)

,narrative
162100,xxxxy xxxxxxxx eaither went business sold comp...
53852,contacted many time failed provide necessary d...
89049,started receiving account statement suntrust t...
29485,victim identity notified collection creditor s...
23310,tried file dispute online equifax continue get...
79532,used direct express debit card corner store at...
55667,filed dispute regard incorrect item credit rep...
76358,sent equifax proof true debt owed xxxxxxxx ord...
112236,looking buy specific breed cat online research...
145282,toyota financial service reported husband cred...


In [31]:
TOKENS = 500
OUTPUT_LENGTH = 45
OUT_DIM = 8
BATCH_SIZE = 128
SAMP = 7
SHUFFLE = 500
UNITS = 50

#Some larger values for Colab
TEST_EPOCHS = 5
TEST_BATCH = 512

if IN_COLAB:
    TEST_EPOCHS = 200
    TEST_BATCH = 1024
    !mkdir weights

### Clean and Build Sequences

We can clean and process the text data with any strategy. Here we'll use a simple strategy of removing punctuation and converting all text to lowercase.

Next we'll use a Keras utility to convert our text into a sequence of words, which we'll then tokenize into integers. The sequence that is created is pretty simple, just a list of all the words in the sentence in the correct order. Basically an OCD version of a normal sentence. 

In [32]:
raw_text = train_text[USE_COL]
## Remove punctuation
raw_text = raw_text.dropna()
raw_text = raw_text.apply(lambda x: x.replace('[{}]'.format(string.punctuation), '').lower())
vocab = set()
sentences = []
for sentence in raw_text:
  current_sentence = text_to_word_sequence(sentence)
  sentences.append(current_sentence)
  vocab.update(current_sentence)
#vocab
#sentences
max_length = max([len(sentence) for sentence in sentences])
sentences[SAMP]

['account',
 'reported',
 'abbreviated',
 'name',
 'full',
 'name',
 'servicer',
 'account',
 'transunion',
 'reporting',
 'date',
 'last',
 'payment',
 'reporting',
 'payment',
 'due',
 'account',
 'time',
 'however',
 'date',
 'inconsistent',
 'however',
 'transunion',
 'reporting',
 'date',
 'last',
 'active',
 'show',
 'show',
 'none',
 'date',
 'match',
 'activity',
 'payment',
 'made',
 'three',
 'bureau',
 'reporting',
 'high',
 'credit',
 'balance',
 'show',
 'transunion',
 'show',
 'amount',
 'violation',
 'failed',
 'addressed',
 'repeatedly',
 'amount',
 'match',
 'information',
 'reported',
 'percent',
 'accurately',
 'according',
 'law',
 'transunion',
 'repeatedly',
 'failed',
 'report',
 'account',
 'dispute',
 'removed',
 'account',
 'dispute',
 'comment',
 'company',
 'reporting',
 'account',
 'top',
 'invalid',
 'account',
 'number',
 'also',
 'listed',
 'authorized',
 'user',
 'considered',
 'account',
 'removed',
 'another',
 'account',
 'reporting',
 'listed',
 'au

#### Tokenize

We can take the text that we split above and encode it as a sequence of integers. We'll then use the tokenizer to convert our sequences into a sequence of integers.

In [33]:
tokenizer = Tokenizer(num_words=TOKENS)
tokenizer.fit_on_texts(sentences)

tokenized_sentences = tokenizer.texts_to_sequences(sentences)
tokenized_sentences[SAMP]

[2,
 56,
 55,
 113,
 55,
 2,
 186,
 6,
 30,
 126,
 5,
 6,
 5,
 28,
 2,
 7,
 136,
 30,
 136,
 186,
 6,
 30,
 126,
 145,
 145,
 30,
 343,
 5,
 36,
 202,
 29,
 6,
 1,
 26,
 145,
 186,
 145,
 38,
 121,
 243,
 38,
 4,
 56,
 382,
 82,
 186,
 243,
 3,
 2,
 18,
 65,
 2,
 18,
 8,
 6,
 2,
 2,
 25,
 23,
 134,
 337,
 2,
 65,
 128,
 2,
 6,
 134,
 337,
 2,
 6,
 1,
 412,
 1,
 412,
 440,
 186,
 6,
 1,
 38,
 1,
 412,
 186,
 6,
 30,
 126,
 6,
 30,
 64,
 120,
 4,
 8,
 30,
 126,
 5,
 186,
 145,
 30,
 126,
 23,
 145,
 30,
 126,
 30,
 126,
 5,
 145,
 30,
 126,
 30,
 126,
 5,
 30,
 8,
 56,
 49,
 23,
 318,
 5,
 2,
 64,
 44,
 131,
 58,
 5,
 235,
 67,
 29,
 6,
 4,
 171,
 2,
 49,
 6,
 260,
 318,
 5,
 282,
 64,
 8,
 145,
 5,
 235,
 136,
 145,
 318,
 5,
 64,
 6,
 260,
 145,
 30,
 126,
 186,
 318,
 5,
 2,
 64,
 136,
 186,
 6,
 30,
 126,
 5,
 145,
 126,
 56,
 30,
 145,
 186,
 145,
 4,
 73,
 56,
 202,
 29,
 200,
 4,
 149,
 147,
 301,
 383,
 338,
 8,
 119,
 3,
 4,
 185,
 1,
 95,
 141,
 23,
 6,
 1,
 412,
 186,
 6,
 6,


### Construct Training Sequences

We are going to be a little slack in the construction of the datasets for training because we are limited in the amount of resources we can handle. All of the datasets that are fed to the model need to be the same length, so we'll set a cap and trucate it here for resource concerns. Our dataset will be constructed as:
<ul>
<li> A sequence of (up to) 24 words as the X data. 
<li> The next word as the Y data.
</ul>

So each sequence is effectively one set of features, and its target is the next word. If we were doing this in reality, we'd want to prep more records from our sample:
<ul>
<li> Suppose a sample sentence is "The quick brown fox jumps over the lazy dog". Our ideal data would have something like:
    <ul> 
    <li>X = "the quick brown", Y = "fox"
    <li> X = "quick brown fox", Y = "jumps"
    <li> X = "brown fox jumps", Y = "over"
    </ul>
</ul>

This is superior both because we are generating much more data to train the model and because we are training the model to predict words in all different positions in the sentence. We'd be predicting almost every word in the training dataset. The words that frequently end a sentence are not necessarily the same as the words that start a sentence or sit in the middle, so making predictions up and down the text will likely lead to a more useful model.

<b>We'd end up with a better model if we generated more sequences from our data and/or added more data. The resource demands make that tough, so we have cut some corners that are easy to remedy in a real-world scenario.</b>

In [34]:
# Find longest sequence
max_real_length = max([len(sentence) for sentence in tokenized_sentences])
print("Longest real sequence:", max_real_length)
if max_real_length > OUTPUT_LENGTH:
  extra_sequences = [s_[OUTPUT_LENGTH:] for s_ in tokenized_sentences]
  trunc_token_sequences = [t_[:OUTPUT_LENGTH] for t_ in tokenized_sentences]
trunc_token_sequences = trunc_token_sequences + extra_sequences
trunc_token_sequences[SAMP]

Longest real sequence: 1548


[2,
 56,
 55,
 113,
 55,
 2,
 186,
 6,
 30,
 126,
 5,
 6,
 5,
 28,
 2,
 7,
 136,
 30,
 136,
 186,
 6,
 30,
 126,
 145,
 145,
 30,
 343,
 5,
 36,
 202,
 29,
 6,
 1,
 26,
 145,
 186,
 145,
 38,
 121,
 243,
 38,
 4,
 56,
 382,
 82]

#### Pad Sequences

We need to pad our sequences so that they are all the same length, as our neural networks require that. The pad_sequences utility does just that, it will fill 0s at either the beginning or end of the sequence, depending on the "padding" option, and make everything the same length. We want to pad before the real data, because we are always planning on predicting the next token, so we want to work from the last value. 

In [35]:
padded_trunc_token_sequences = pad_sequences(trunc_token_sequences, maxlen=OUTPUT_LENGTH, padding='pre')
padded_trunc_token_sequences[SAMP]

array([  2,  56,  55, 113,  55,   2, 186,   6,  30, 126,   5,   6,   5,
        28,   2,   7, 136,  30, 136, 186,   6,  30, 126, 145, 145,  30,
       343,   5,  36, 202,  29,   6,   1,  26, 145, 186, 145,  38, 121,
       243,  38,   4,  56, 382,  82], dtype=int32)

#### Write Tokenized Data to Disk

We can write the tokenized data to disk so that we can use it later. This will save us from having to redo that step that is slow. Since we chopped our data size down a bit, this isn't super needed. I tried to generate all of the sequences noted above and I both exceeded the Colab memory limits and it took a while to run. I wouldn't want to repeat that if I can avoid it, and hard drive space is cheap, so writing the interim results to disk is a good work around. In real scenarios when we had massive amounts of data we would need to load it incrementally from disk anyway, so this is a free win.

In [36]:
#token_path = 'data/sample_tokenized_sentences.csv'
token_path = 'data/padded_sample_tokenized_sentences.csv'
if IN_COLAB:
  !mkdir data
  
if os.path.exists(token_path):
    df_prepped = pd.read_csv(token_path, header=None)
    prepped_sentences = np.array(df_prepped.values.tolist())
else:
    df_prepped = pd.DataFrame(padded_trunc_token_sequences)
    df_prepped.to_csv(token_path, header=False, index=False)

#### Dataset and Array

This data can be reasonably used either as a regular array or or a tensorflow dataset. We'll use both here, just to show that it can be done either way. If we we using the full set of sequences the data would be quite a bit larger with all the fully padded out sequences, we'd probably need to write the data to disk and load it with a dataset or generator. 

In [37]:
print(padded_trunc_token_sequences.shape)
y_t = padded_trunc_token_sequences[:, -1].reshape(-1, 1)
y_t = ku.to_categorical(y_t, num_classes=TOKENS)
print("Y shape:", y_t.shape)
X_t = padded_trunc_token_sequences[:, :-1]
print("X shape:", X_t.shape)

(324822, 45)
Y shape: (324822, 500)
X shape: (324822, 44)


### Model

Now we model. The data that we made mirrors the construction of a sentence.
<ul>
<li> X features - the sentence up to this point. 
<li> Y target - the word(s) that should come next. 
</ul>

So, the model is effectively working to generate text just like a time series model works to predict the next value in a sequence of stock prices or hourly temperature. We train the model on, hopefully a large number of senteneces, where is sees many examples of "here are some words" (X values) and "here is the next word" (Y value). If we give it lots and lots of that training data, it should become better and better at determining what should come next, given the existing sentence. 

To do this well, we'd need a lot more data than we have, and much more time to train. We'd want to give the model enough data so that it can see lots and lots of examples of the same word in different contexts, and of similar contexts with different words. The patterns of language are really complex, so we need data that provides enough variation to demonstrate the patterns. <b>The actual performance of our model at creating sensible text will be bad, we should not get our hopes up, but the process is sound.</b>

#### Embedding Layer

We also use an embedding layer here, which accepts our integer inputs and converts them to a vector of a specified size. This is a way of representing the words in a way that is more useful for the model. We saw embeddings with word2vec during the NLP portions. Just as it did then, embedding will represent each of our tokens as a vector of a specified size, or as a value in N dimensional space. 

![Embedding](images/embedding.png "Embedding")

When we check the summary of the model we can see that the embedding layer has a lot of parameters, this is because it is learning the vector representation of each of the words in our vocabulary. When we used word2vec we used a pretrained model - the N dimensional space was already defined, and we placed our words into it. Here, we are letting the model learn the space and the vectors that represent the words, so the vector representation of each word is learned directly from the data. If we are dealing with a scenarion where we have a lot of data and a specialized vocabulary (such as in industry) this can be very useful. The model will learn which words are similar and which are not, based on the context of the text provided in training.

If we consider embedding from the perspective of the actual prediction, it also makes a lot of sense that this is a prefered approach. We are not generally trying to predict the exact word, as we would with a classification that chooses the correct answer from a list of options. When dealing with language we are more likely trying to predict the next <i>type</i> of word - both if it is a noun/verb/etc and the meaning. Embedding the words allows us to use that multidimensional representation aim for words that are similar. When dealing with generative models with large vocabularies our model will follow "Do you want to go out sometime and grab a ________" with something like "beer" or "coffee" which are similar in this context, and close in their embedding representation. Using embeddings, the loss that is calculated each round is measuring the distance between the predicted word and the actual word in N dimensions, so grabbing words that are very close, or similar, is expected and desired. 

#### Let's Go Bi

For this model we'll use a bidirectional LSTM layer. This is a layer that will process the input sequence in both directions, so it will see the sequence from the beginning and from the end. This is useful because it allows the model to consider what should come next, but also what should come before, which can be useful for determining the next word. The idea of a bidirectional layer is that if our data is in a sequence we can feed it through normally, but we can also reverse it. Doing this gives the model a differnet view on the data, and especially in cases like language creation, where the meaning of the output can depend on the entirety of the sequence. 

![Bidirectional LSTM](images/bidirectional.webp "Bidirectional LSTM")

Bidirectional layers are most prevalent in NLP, as adding the ability to look at the sequence of words in both directions can really help models build a better understanding of the context of the words. There is a speed penalty due to each layer doing roughly double the number of calculations, but for language tasks such as this, it is pretty likely to be worth it, particularly if we were to expand the dataset. Bidirectional layers can be added simply, as below; on the Keras documentation there are a few more details, mainly that the forward and backward layers can be configured separately then combined. When using a bidirectional layer, one option that we should commonly pay attention to is the "merge_mode" option, which determines how the forward and backward layers are combined to produce the final output. The default is "concat" which combines the outputs by concatenating them, but there are other options such as "sum" or "ave" which can be useful in some cases - hyperparameter tuning is the way to find the "best" option. We will use the default here, and can sidestep the other details of bidirectional layers for now.

In [38]:
model = Sequential()
model.add(Embedding(input_dim=TOKENS, output_dim=OUT_DIM, input_length=OUTPUT_LENGTH-1))
#model.add(LSTM(UNITS, return_sequences=True))
#model.add(LSTM(UNITS, return_sequences=True))
#model.add(LSTM(UNITS))
model.add(Bidirectional(LSTM(UNITS, return_sequences=True)))
model.add(Bidirectional(LSTM(UNITS, return_sequences=True)))
model.add(Bidirectional(LSTM(UNITS)))
model.add(Flatten())
model.add(Dense(TOKENS, activation='softmax'))

model.compile(optimizer='rmsprop',loss='categorical_crossentropy',metrics=['acc'])
print(model.summary())

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 44, 8)             4000      
                                                                 
 bidirectional_3 (Bidirectio  (None, 44, 100)          23600     
 nal)                                                            
                                                                 
 bidirectional_4 (Bidirectio  (None, 44, 100)          60400     
 nal)                                                            
                                                                 
 bidirectional_5 (Bidirectio  (None, 100)              60400     
 nal)                                                            
                                                                 
 flatten_1 (Flatten)         (None, 100)               0         
                                                      

In making this, lots of epochs definately gave better results, as did a swap to bidirectional layers. I saw this in both the raw accuracy scores and the relative sense of the generated text. 

In [40]:
# Try with dataframes
early_stop = EarlyStopping(monitor='loss', patience=50)
save_weights = tf.keras.callbacks.ModelCheckpoint('weights/lstm_gen_weights.h5', save_best_only=True, monitor='loss', mode='min', save_weights_only=True)

history = model.fit(X_t, y_t, batch_size=TEST_BATCH, epochs=TEST_EPOCHS, verbose=1, callbacks=[early_stop, save_weights])
plot_acc(history)
plot_loss(history)

Epoch 1/5
635/635 [==============================] - 542s 835ms/step - loss: 3.9244 - acc: 0.3266
Epoch 2/5
635/635 [==============================] - 480s 756ms/step - loss: 3.6631 - acc: 0.3486
Epoch 3/5
635/635 [==============================] - 570s 897ms/step - loss: 3.5028 - acc: 0.3746
Epoch 4/5
635/635 [==============================] - 707s 1s/step - loss: 3.3980 - acc: 0.3971
Epoch 5/5
300/635 [=============>................] - ETA: 8:21 - loss: 3.3405 - acc: 0.4099

KeyboardInterrupt: 

If you haven't changed anything, this will pick up where the training above left off, we really only need one or the other, but we're getting wild and crazy here. If you're running this locally, one or the other will probably do the job. 

In [ ]:
train_ds = tf.data.Dataset.from_tensor_slices((X_t, y_t)).shuffle(SHUFFLE).batch(TEST_BATCH).prefetch(tf.data.experimental.AUTOTUNE)

In [ ]:
# Or with datasets...
history = model.fit(train_ds, epochs=TEST_EPOCHS, verbose=1, callbacks=[early_stop, save_weights])
plot_acc(history)
plot_loss(history)

Epoch 1/5
49/49 [==============================] - 28s 554ms/step - loss: 5.3921 - acc: 0.0538
Epoch 2/5
49/49 [==============================] - 28s 567ms/step - loss: 5.4334 - acc: 0.0538
Epoch 3/5
49/49 [==============================] - 25s 516ms/step - loss: 5.4142 - acc: 0.0538
Epoch 4/5
49/49 [==============================] - 26s 534ms/step - loss: 5.4069 - acc: 0.0538
Epoch 5/5
49/49 [==============================] - 26s 530ms/step - loss: 5.3859 - acc: 0.0563


### Generate Some Text

We can generate text once the model is trained. We'll start with a seed sentence, and then we'll use the model to predict the next word. We'll then append that word to the sentence and use the model to predict the next word, and so on. There is a little helper function to do this for us with limited repetition.

Another new thing is that we create an inverse dictionary to map the encoded words back to the original words.

### Temperature

One weirdly named factor that is important in text generation is the temperature. The temperature is a factor that we can use to control the randomness of the output. To generate text, we are essentially using a probability distribution to determine what the next word should be - the softmax output of the model will tell us the most likely next word. The issue is that certain words are way more likely than others - "the", "it", "a", "and", etc. are all very common words so we can expect the model to predict them as "most likely" a lot, probably too often. 

![Temperature](images/temperature.gif "Temperature")

The most direct way to combat this is to add some degree of randomness to which word we select - we'll still pick the most likely word more often than any other, but we'll also pick other words that have some degree of likelihood at random. The higher the temperature the more randomness is introduced. A correct value requires tuning with human feedback, and it'll vary depending on the base quality of the model - large models that are trained on huge volumes of text and are deep enough to pick up on the "what type of word should be here" patterns will be able to generate better text with lower temperatures. Our model here is small and kind of sucks, so the temperature needs to be higher to get anything remotely usable. The implementation here is stolen shamelessly from the internet, the details don't really matter all that much, we just need to vary our predictions away from always simply picking the most likely word.

In [ ]:
inverse_dict = {v: k for k, v in tokenizer.word_index.items()}

In [ ]:
from random import *

def sampleWord(preds, temperature=1.0):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds.flatten(), 1)
    return np.argmax(probas)

def generate_text(seed_text, next_words, model, length_max, tok, inverse, temperature=1.0):
    out = seed_text
    for _ in range(next_words):
        token_list = tok.texts_to_sequences([out])[0]
        token_list = pad_sequences([token_list], maxlen=length_max-1, padding='pre')
        #print(token_list)
        predicted = model.predict(token_list, verbose=0)
        #print(tok.sequences_to_texts(predicted))
        word = inverse[sampleWord(predicted, temperature=temperature)]
        out += " "+word
        #print(out)
    return out

#### Fake Text Time!

<b>Note:</b> we haven't built in any error handling, so it is possible to get random errors with unknown seeds. 

In [ ]:
print (generate_text("Last night I went on a date with", next_words=30, model=model, length_max=OUTPUT_LENGTH, tok=tokenizer, inverse=inverse_dict), "\n")
print (generate_text("We are going to the ", next_words=30, model=model, length_max=OUTPUT_LENGTH, tok=tokenizer, inverse=inverse_dict), "\n")
#print (generate_text("Where are all of the", next_words=30, model=model, length_max=OUTPUT_LENGTH, tok=tokenizer, inverse=inverse_dict), "\n")
#print (generate_text("I am scared", next_words=30, model=model, length_max=OUTPUT_LENGTH, tok=tokenizer, inverse=inverse_dict), "\n")
print (generate_text("School is out for", next_words=30, model=model, length_max=OUTPUT_LENGTH, tok=tokenizer, inverse=inverse_dict), "\n")

Last night I went on a date with not the days the you and strong of can fuck i’m do will there to me on know value it way almost not its a guys comments x200b b 3 

We are going to the  them anything they i as new options money the 1 change and width not and already is some auto retards weeks have life for my then chart and apes and 

Where are all of the apes it to its in both up a on but now over why low potential auto money really pretty if done at total know his with and shit a market 

I am scared have of stock t it and take as a holding was 0 great a people t well lot help a moon be s gme no stocks keep been time to 

School is out for bad to that in format end r what stock big time by covid i 3 that the a here after in time of can't day shorts be end and s 



A few long ones... as with a time series, we shouldn't be surprised if the text becomes worse the farther we go from 

In [ ]:
print (generate_text("Last night I went on a date with", next_words=100, model=model, length_max=OUTPUT_LENGTH, tok=tokenizer, inverse=inverse_dict), "\n")
print (generate_text("When is work happening", next_words=100, model=model, length_max=OUTPUT_LENGTH, tok=tokenizer, inverse=inverse_dict), "\n")
print (generate_text("The Toronto Raptors won and Rob Ford's body rose from the dead", next_words=100, model=model, length_max=OUTPUT_LENGTH, tok=tokenizer, inverse=inverse_dict), "\n")

Last night I went on a date with go their like the the months until than 2021 100 term on i in the the or the today may put platform in with however s and was why are i 15 https a still hold no at hold it so term advice if a them please 2 my today investor hedge great huge all though t t the webp s 4 moon may right back still could profit and back strong the the lose put trading 8 there png good that's 13 every 5 i'm go high in is here public t after selling on 13 am the use 



KeyError: 0

All in all, this isn't really that much worse than actual wallstreetbets posts. Also, in one run while making this my model generated a rocket ship emoji, because we are going to the moon. Emojis are just weird character sequences, so that makes sense, but I had never really considered that as a possibility before. 

And if we make the temperature low, we likely get worse results.

In [ ]:
print (generate_text("Last night I went on a date with", next_words=30, model=model, length_max=OUTPUT_LENGTH, tok=tokenizer, inverse=inverse_dict, temperature=.1), "\n")
print (generate_text("We are going to the ", next_words=30, model=model, length_max=OUTPUT_LENGTH, tok=tokenizer, inverse=inverse_dict, temperature=.1), "\n")

Last night I went on a date with gme the moon stock stock gme the moon stock stock stock the moon stock stock gme gme gme of the of the of the of the of the of the 

We are going to the  moon moon gme stock stock stock stock the stock stock the moon gme gme the moon stock stock stock of the of the of the of the of the of 



### Homemade Text Generation Results

The LSTM model works well for text generation, if we had more data, more time, and spent a bit more effort cleaning up the edge cases in our code here we could likely get something pretty legible. 

The biggest difference between this and the impressive larger models is that they are higher capacity, trained with more data, and sometimes tuned with human feedback. These things combine to make the model more likely to pick up on the patterns of language and generate text that is more likely to be grammatically correct and follow the expected flow of language. Our model is really struggling to just find a reasonable word to use next, we haven't allowed it to learn enough to really construct logical sentences. 

For our next trick, we can borrow the power of gpt-2...